In [1]:
import os
import gc
import re
import sys
import json
import time
import random
import requests
import argparse
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
import scipy.io as sio
from tqdm import tqdm
from pathlib import Path
from collections import Counter
from collections import defaultdict
from logging import getLogger
from ast import Global
from functools import reduce
from trd import *
from ALDI import *
from utils import *
sys.path.append('/root/linghui/drs/TRD-main/daisyRec/')
import daisy
from daisy.utils.loader import RawDataReader, Preprocessor
from daisy.utils.splitter import TestSplitter, ValidationSplitter
from daisy.utils.config import init_seed, init_config, init_logger
from daisy.utils.metrics import MAP, NDCG, Recall, Precision, HR, MRR
from daisy.utils.sampler import BasicNegtiveSampler, SkipGramNegativeSampler, UniqueNegativeSampler
from daisy.utils.dataset import AEDataset, BasicDataset, CandidatesDataset, get_dataloader
from daisy.utils.utils import get_history_matrix, get_ur, build_candidates_set, ensure_dir, get_inter_matrix
from daisy.model.MFRecommender import MF
from daisy.model.FMRecommender import FM
from daisy.model.NFMRecommender import NFM
from daisy.model.NGCFRecommender import NGCF
from daisy.model.EASERecommender import EASE
from daisy.model.SLiMRecommender import SLiM
from daisy.model.VAECFRecommender import VAECF
from daisy.model.NeuMFRecommender import NeuMF
from daisy.model.PopRecommender import MostPop
from daisy.model.KNNCFRecommender import ItemKNNCF
from daisy.model.PureSVDRecommender import PureSVD
from daisy.model.Item2VecRecommender import Item2Vec
from daisy.model.LightGCNRecommender import LightGCN
from daisy.utils.metrics import calc_ranking_results

/opt/conda/lib/python3.8/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.4)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [2]:
model_config = {
    'mostpop': MostPop,
    'slim': SLiM,
    'itemknn': ItemKNNCF,
    'puresvd': PureSVD,
    'mf': MF,
    'fm': FM,
    'ngcf': NGCF,
    'neumf': NeuMF,
    'nfm': NFM,
    'multi-vae': VAECF,
    'item2vec': Item2Vec,
    'ease': EASE,
    'lightgcn': LightGCN,
}


config = init_config()
init_seed(config['seed'], config['reproducibility'])
init_logger(config)
logger = getLogger()
logger.info(config)
config['logger'] = logger

trd_version = 'new_flag_reg'
# trd_version = 'efficient_v1'
save_path = config['save_path'] + config['version']
ensure_dir(save_path)

file_path = save_path + f'{config["dataset"]}/'
ensure_dir(file_path)

saved_data_path = config['save_path'] + f'daisy_1/{config["dataset"]}/' + 'data/'
ensure_dir(saved_data_path)

saved_result_path = file_path + f'{config["algo_name"]}/'
ensure_dir(saved_result_path)

saved_model_path = saved_result_path + 'model/'
ensure_dir(saved_model_path)

saved_trd_path = saved_result_path + f'{trd_version}/'
ensure_dir(saved_trd_path)

saved_rec_path = saved_result_path + 'rec_list/'
ensure_dir(saved_rec_path)

saved_metric_path = saved_result_path + 'metric/'
ensure_dir(saved_metric_path)
config['res_path'] = saved_metric_path

ui_num = np.load(saved_data_path + 'ui_cate.npy')
config['user_num'] = ui_num[0]
config['item_num'] = ui_num[1]
# config['cate_num'] = ui_num[2]
print(f"user number: {config['user_num']}  item number: {config['item_num']}")

23 Jan 09:57 INFO - {'version': 'daisy_1/', 'trd_version': 'drs_trd', 'gpu': '1', 'seed': 2024, 'reproducibility': True, 'state': None, 'optimization_metric': 'ndcg', 'hyperopt_trail': 20, 'tune_testset': False, 'tune_pack': '{"lr": [0.001, 0.005, 0.01], "reg_1": [0.000001, 0.00001, 0.0001, 0.001, 0], "reg_2": [0.000001, 0.00001, 0.0001, 0.001, 0], "num_layers": [2, 3, 4]}', 'algo_name': 'mf', 'data_path': '/data/linghui/', 'save_path': '/data/linghui/drs/', 'res_path': None, 'dataset': 'steam', 'val_method': 'tsbr', 'test_method': 'tsbr', 'fold_num': 1, 'val_size': 0.1, 'test_size': 0.2, 'topk': 50, 'n_actions': 20, 'cand_num': 1000, 'sample_method': 'uniform', 'sample_ratio': 0, 'num_ng': 4, 'batch_size': 1024, 'loss_type': 'BPR', 'init_method': 'default', 'optimizer': 'default', 'early_stop': True, 'content_dim': 328, 'train_step': 120, 'prepro': '10filter', 'level': 'u', 'positive_threshold': 1.0, 'UID_NAME': 'user', 'IID_NAME': 'item', 'INTER_NAME': 'label', 'TID_NAME': 'timestamp

user number: 79216  item number: 14258


In [3]:
train_total_set = pd.read_csv(saved_data_path + 'train_total_set.csv', index_col=0)
test_u_all = np.load(saved_data_path + 'test_u_all.npy', allow_pickle=True)
test_ucands_all = np.load(saved_data_path + 'test_ucands_all.npy',allow_pickle=True)
test_ur_all = np.load(saved_data_path + 'test_ur_all.npy', allow_pickle=True)
data_last_stage = train_total_set
# result_df_all = pd.DataFrame()
warm_item_list_stage = []
cold_item_list_stage = []
new_item_list_stage = []
for stage in range(config['test_stage']):
    logger.info(f'begin test stage {stage}')
    # load test data
    test_u = test_u_all[stage]
    test_ur = test_ur_all[stage]
    test_ucands = test_ucands_all[stage]
    test_set = pd.read_csv(saved_data_path + f'test_set_{stage}.csv')
    # test_pd = pd.read_csv(saved_data_path + f'test_df_{stage}', index_col = 0)

    # update warm ，cold set and new items in this stage
    start_time = time.time()
    warm_item_list, cold_item_list = update_new_item(data_last_stage, config)
    warm_item_list_stage.append(warm_item_list)
    cold_item_list_stage.append(cold_item_list)
    # config['warm_item_list'] = warm_item_list
    # config['cold_item_list'] = cold_item_list
    new_item_list = list(set(test_set[config['IID_NAME']].unique()) - set(warm_item_list))
    new_item_list_stage.append(new_item_list)
    config['topk_list'] = [10, 20, 50]
    data_last_stage = pd.concat([data_last_stage, test_set])
    end_time = time.time()
    elapsed_time = end_time - start_time
    logger.info(f":finished update warm/cold_list: {elapsed_time:.6f} s")

23 Jan 09:57 INFO - begin test stage 0
23 Jan 09:57 INFO - :finished update warm/cold_list: 0.075160 s
23 Jan 09:57 INFO - begin test stage 1
23 Jan 09:57 INFO - :finished update warm/cold_list: 0.129882 s
23 Jan 09:57 INFO - begin test stage 2
23 Jan 09:57 INFO - :finished update warm/cold_list: 0.110933 s
23 Jan 09:57 INFO - begin test stage 3
23 Jan 09:57 INFO - :finished update warm/cold_list: 0.135777 s
23 Jan 09:57 INFO - begin test stage 4
23 Jan 09:57 INFO - :finished update warm/cold_list: 0.150264 s


In [5]:
np.save(saved_data_path + 'warm_item_list_stage.npy', warm_item_list_stage)
np.save(saved_data_path + 'cold_item_list_stage.npy', cold_item_list_stage)
np.save(saved_data_path + 'new_item_list_stage.npy', new_item_list_stage)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (5,) + inhomogeneous part.

In [4]:
def set_values_by_id(interaction_history, item_num):
    """
    将用户的交互历史（列表）转换为一个固定长度的列表，交互过的物品位置为 1，未交互过的为 0。
    """
    # 使用 numpy 的向量化操作替代循环
    return np.isin(np.arange(item_num), interaction_history).astype(int).tolist()

def get_weight(warm_item_list, cold_item_list):
    """
    计算暖项目和冷项目的权重
    """
    warm_num = len(warm_item_list)
    warm_weight = np.arange(warm_num, 0, -1)  # 从 warm_num 到 1

    cold_num = len(cold_item_list)
    if cold_num > 1:
        cold_weight = 1 + np.arange(cold_num) * (warm_num - 1) / (cold_num - 1)
    else:
        cold_weight = np.array([1])
    return warm_weight, cold_weight

def get_user_hist_tgf(exp_list, warm_item_list, cold_item_list, warm_weight, cold_weight):
    """
    计算用户的分类权重差异（tgf）
    """
    if np.sum(exp_list) == 0:
        return 0

    # 归一化经验值
    exp_list = exp_list / np.sum(exp_list)

    # 提取暖项目和冷项目的经验值
    warm_exp_list = exp_list[warm_item_list]
    cold_exp_list = exp_list[cold_item_list]

    # 计算暖项目和冷项目的部分
    warm_part = np.sum(warm_exp_list * warm_weight) / len(warm_item_list)
    cold_part = np.sum(cold_exp_list * cold_weight) / len(cold_item_list)

    # 计算分类的权重差异
    user_tgf = warm_part - cold_part
    if user_tgf < 0:
        user_tgf = user_tgf / (len(warm_item_list) / len(cold_item_list))
    return user_tgf

def calculate_nc(input_set, target_set):
    """
    统计给定集合中有多少比例的值出现在目标集合中

    参数:
    input_set: 输入的集合
    target_set: 目标集合

    返回:
    proportion: 出现在目标集合中的比例
    """
    if not input_set:
        return 0.0
    return len(set(input_set) & set(target_set)) / len(input_set)

def process_user_interaction_data(train_rl_set, warm_item_list, cold_item_list, config):
    """
    处理用户交互数据，生成包含用户历史、经验列表、分类权重差异和冷项目比例的 DataFrame。

    参数:
    - train_rl_set: 训练集的用户交互数据（DataFrame）
    - warm_item_list: 暖项目列表
    - cold_item_list: 冷项目列表
    - cold_item_set: 冷项目集合
    - config: 配置字典，包含 item_num 等参数

    返回:
    - user_rl_df: 处理后的用户交互数据（DataFrame）
    """
    # 按用户分组并生成用户历史列表
    user_rl_df = train_rl_set.groupby('user')['item'].apply(list).reset_index(name='user_history')

    # 生成用户的经验列表
    item_range = np.arange(config['item_num'])
    user_rl_df['exp_list'] = user_rl_df['user_history'].apply(
        lambda x: np.isin(item_range, x).astype(int).tolist()
    )

    # 计算暖项目和冷项目的权重
    warm_weight, cold_weight = get_weight(warm_item_list, cold_item_list)

    # 计算用户的分类权重差异（tgf）
    user_rl_df['hist_tgf'] = user_rl_df['exp_list'].apply(
        lambda x: get_user_hist_tgf(np.array(x), warm_item_list, cold_item_list, warm_weight, cold_weight)
    )

    # 计算用户的冷项目比例（hist_nc）
    user_rl_df['hist_nc'] = user_rl_df['user_history'].apply(
        lambda x: calculate_nc(x, cold_item_list)
    )

    return user_rl_df

def get_common_users(dataframes):
    """
    从多个 Pandas DataFrame 中提取所有共有的 'user' 列值。
    """
    if not dataframes:
        return set()

    # 将所有 'user' 列的值合并为一个 numpy 数组
    all_users = [df['user'].values for df in dataframes if 'user' in df.columns]

    # 如果没有有效的 DataFrame，返回空集合
    if not all_users:
        return set()

    # 使用 numpy 的 reduce 求交集
    return set(reduce(np.intersect1d, all_users))

In [5]:
test_set_list = []
for stage in range(config['test_stage']):
    test_set = pd.read_csv(saved_data_path + f'test_set_{stage}.csv')
    test_set_list.append(test_set)
# process_user_interaction_data(test_set, warm_item_list, cold_item_list, config)

In [6]:
import numpy as np

def calculate_mse(list1, list2):
    """
    计算两个列表之间的均方误差（MSE）。

    参数:
    - list1: 第一个列表
    - list2: 第二个列表

    返回:
    - mse: 两个列表之间的均方误差
    """
    return np.mean((np.array(list1) - np.array(list2)) ** 2)

def average_mse(array1, array2):
    """
    计算两个数组中一一对应的列表之间的 MSE，并返回归一化后的平均值。

    参数:
    - array1: 第一个数组，包含五个列表
    - array2: 第二个数组，包含五个列表

    返回:
    - avg_mse: 归一化后的五个 MSE 值的平均值（范围 [0, 1]）
    """
    if len(array1) != len(array2):
        raise ValueError("两个数组的长度必须相同")

    # 计算每个对应列表的 MSE
    mse_values = [calculate_mse(list1, list2) for list1, list2 in zip(array1, array2)]

    # 归一化 MSE 值
    normalized_mse_values = [mse / 4 for mse in mse_values]

    # 计算归一化后的 MSE 平均值
    avg_mse = np.mean(normalized_mse_values)
    return avg_mse

In [7]:
common_users = get_common_users(test_set_list)

In [18]:
len(common_users)

17286

In [10]:
np.save(saved_data_path + 'common_users.npy', common_users)

In [8]:
#calculate ground truth
start_time = time.time()
mode = 'ground_truth'
hist_list = []
for stage in range(config['test_stage']):
    df = test_set_list[stage]
    df = df[df['user'].isin(common_users)]
    warm_item_list = warm_item_list_stage[stage]
    cold_item_list = cold_item_list_stage[stage]
    df_processed = process_user_interaction_data(df, warm_item_list, cold_item_list, config)
    df_processed.to_csv(saved_data_path + f'unf_{mode}_stage{stage}')
    hist_list.append(df_processed['hist_tgf'].to_list())
end_time = time.time()
elapsed_time = end_time - start_time
print(f":finished update warm/cold_list: {elapsed_time:.6f} s")

:finished update warm/cold_list: 6.962107 s


In [12]:
elapsed_time

276.33456230163574

In [13]:
np.save(saved_data_path + 'hist_list_ground_truth.npy', hist_list)

In [8]:
hist_list = np.load(saved_data_path + 'hist_list_ground_truth.npy')

In [14]:
config['test_stage']

5

In [17]:
len(hist_list[0])

17286

In [38]:
# mf + light_gcn
start_time = time.time()
for method in ['backbone', 'pd', 'pearson', 'cnif']:
    print(saved_metric_path)
    config['debias_method'] = method
    user_hist_list = []
    for stage in range(config['test_stage']):
        df = pd.read_csv(saved_rec_path + f"{config['debias_method']}_{config['dataset']}_{config['algo_name']}_rec_df{stage}")
        df = df[df['user'].isin(common_users)]
        warm_item_list = warm_item_list_stage[stage]
        cold_item_list = cold_item_list_stage[stage]
        df_processed = process_user_interaction_data(df, warm_item_list, cold_item_list, config)
        df_processed.to_csv(saved_metric_path + f"unf_{config['debias_method']}_stage{stage}")
        user_hist_list.append(df_processed['hist_tgf'].to_list()) 
    np.save(saved_metric_path + f"hist_list_{config['debias_method']}.npy", user_hist_list)
end_time = time.time()
elapsed_time = end_time - start_time
print(f":finished update warm/cold_list: {elapsed_time:.6f} s")

/data/linghui/drs/daisy_1/steam/lightgcn/metric/
/data/linghui/drs/daisy_1/steam/lightgcn/metric/
/data/linghui/drs/daisy_1/steam/lightgcn/metric/
/data/linghui/drs/daisy_1/steam/lightgcn/metric/
:finished update warm/cold_list: 1142.615058 s


In [21]:
elapsed_time

1141.8149898052216

In [25]:
df.user.nunique()

6192

In [22]:
np.shape(user_hist_list)

(5, 17286)

In [40]:
result_list = []
for method in ['backbone', 'pd', 'pearson', 'cnif']:
    config['debias_method'] = method
    user_hist_ground_truth = np.load(saved_data_path + 'hist_list_ground_truth.npy', allow_pickle=True) 
    user_hist_baseline = np.load(saved_metric_path + f"hist_list_{config['debias_method']}.npy", allow_pickle=True) 
    avg_mse = average_mse(user_hist_ground_truth, user_hist_baseline)
    result_list.append(avg_mse)
result_list

[0.06209650499226749,
 0.06728811082521705,
 0.0675830702855251,
 0.06079082819450367]

In [40]:
result_list

[0.10197813356697974,
 0.12413935504751628,
 0.12486927859711269,
 0.12311311027588649]

In [11]:
# mf/lightgcn + fairagent
# mf + light_gcn
start_time = time.time()
user_hist_list = []
# rec_results_fair_alpha2.0_beta0.5_gama0.1_stage0.csv
for stage in range(config['test_stage']):
    df = pd.read_csv(saved_metric_path + f"fairagent_rec_results_fair_alpha0.0_beta0.0_gama0.1_stage{stage}.csv")
    df = df[df['user'].isin(common_users)]
    warm_item_list = warm_item_list_stage[stage]
    cold_item_list = cold_item_list_stage[stage]
    df_processed = process_user_interaction_data(df, warm_item_list, cold_item_list, config)
    df_processed.to_csv(saved_metric_path + f"fairagent_unf_stage{stage}")
    user_hist_list.append(df_processed['hist_tgf'].to_list()) 
np.save(saved_metric_path + f"fairagent_hist_list.npy", user_hist_list)
end_time = time.time()
elapsed_time = end_time - start_time
print(f":finished update warm/cold_list: {elapsed_time:.6f} s")

user_hist_ground_truth = np.load(saved_data_path + 'hist_list_ground_truth.npy') 
user_hist_baseline = np.load(saved_metric_path + f"fairagent_hist_list.npy") 
avg_mse = average_mse(user_hist_ground_truth, user_hist_baseline)
avg_mse

:finished update warm/cold_list: 286.855894 s


0.04248475644920431

In [14]:
avg_mse

0.02395332419483007

In [ ]:
print('done')

In [31]:
#ALDI
start_time = time.time()
for method in ['backbone', 'pd', 'pearson', 'cnif']:
    print(saved_metric_path)
    config['debias_method'] = method
    user_hist_list = []
    for stage in range(config['test_stage']):
        # rec_results_aldi_backbone_stage0.csv
        df = pd.read_csv(saved_metric_path + f"rec_results_aldi_{config['debias_method']}_stage{stage}.csv")
        df = df[df['user'].isin(common_users)]
        warm_item_list = warm_item_list_stage[stage]
        cold_item_list = cold_item_list_stage[stage]
        df_processed = process_user_interaction_data(df, warm_item_list, cold_item_list, config)
        df_processed.to_csv(saved_metric_path + f"ALDI_unf_{config['debias_method']}_stage{stage}")
        user_hist_list.append(df_processed['hist_tgf'].to_list()) 
    np.save(saved_metric_path + f"ALDI_hist_list_{config['debias_method']}.npy", user_hist_list)
end_time = time.time()
elapsed_time = end_time - start_time
print(f":finished update warm/cold_list: {elapsed_time:.6f} s")

/data/linghui/drs/daisy_1/steam/mf/metric/
/data/linghui/drs/daisy_1/steam/mf/metric/
/data/linghui/drs/daisy_1/steam/mf/metric/
/data/linghui/drs/daisy_1/steam/mf/metric/
:finished update warm/cold_list: 1161.540070 s


In [ ]:
np.shape(user_hist_list)

(5, 17286)

In [ ]:
result_list = []
for method in ['backbone', 'pd', 'pearson', 'cnif']:
    config['debias_method'] = method
    user_hist_ground_truth = np.load(saved_data_path + 'hist_list_ground_truth.npy') 
    user_hist_baseline = np.load(saved_metric_path + f"ALDI_hist_list_{config['debias_method']}.npy") 
    avg_mse = average_mse(user_hist_ground_truth, user_hist_baseline)
    result_list.append(avg_mse)
result_list

[0.045395306345969486,
 0.040035236661726564,
 0.03392357244545219,
 0.025276180560409135]

In [72]:
np.max(user_hist_baseline)

0.7018536273569832

In [71]:
np.max(user_hist_ground_truth)

0.33385620218463186

In [34]:
#ALDI + fairagent
start_time = time.time()
user_hist_list = []
for stage in range(config['test_stage']):
    # rec_results_aldi_backbone_stage0.csv
    df = pd.read_csv(saved_metric_path + f"ALDI_rec_results_fair_alpha2.0_beta0.2_gama0.1_stage{stage}.csv")
    df = df[df['user'].isin(common_users)]
    warm_item_list = warm_item_list_stage[stage]
    cold_item_list = cold_item_list_stage[stage]
    df_processed = process_user_interaction_data(df, warm_item_list, cold_item_list, config)
    df_processed.to_csv(saved_metric_path + f"ALDI_fairagent_unf_stage{stage}")
    user_hist_list.append(df_processed['hist_tgf'].to_list()) 
np.save(saved_metric_path + f"ALDI_fairagent_hist_list.npy", user_hist_list)
end_time = time.time()
elapsed_time = end_time - start_time
print(f":finished update warm/cold_list: {elapsed_time:.6f} s")

:finished update warm/cold_list: 285.194105 s


In [36]:
user_hist_ground_truth = np.load(saved_data_path + 'hist_list_ground_truth.npy') 
user_hist_baseline = np.load(saved_metric_path + f"ALDI_fairagent_hist_list.npy") 
avg_mse = average_mse(user_hist_ground_truth, user_hist_baseline)
avg_mse

0.040556648009419015

In [ ]:
ALDI_rec_results_fair_alpha2.0_beta0.5_gama0.1_stage0.csv